## Segmenting and Clustering Neighborhoods in Toronto ##

### Part 1 - Applied Data Science Capstone Project ###

In this part we rescue a table from Wikipedia with PostalCode, Borough, and Neighborhood information of Toronto.

In [1]:
# Import main libraries for this project.

import pandas as pd
import numpy as np